#### 简单线性回归

简单线性回归(单变量univariate)的目标是建立单个特征(explanatory variable, $x$)与联系值目标(response variable, $y$)之间的线性关系。具有一个解释变量的线性模型方程定义如下：

$$y = w_{1}x+ b$$

其中，$w_{1}$ 是权重(weight)，$b$ 是偏置单元(bias unit)。前者为解释变量的权重，后者为y轴截距(intercept)。因此，线性回归可以理解为通过训练样本寻找最佳拟合直线，如下图：

